In [1]:
import math
import numpy as np
import numpy.random as npr
from numpy import linalg
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset
import torch.optim as optim
import time
from math import *
torch.cuda.set_device(0)

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
class ResNet(nn.Module):
    def __init__(self,dim, m):
        super(ResNet, self).__init__()
        self.Ix = torch.zeros([dim,m]).cuda()
        self.Ix[0,0] = 1
        self.Ix[1,1] = 1
        self.fc1 = nn.Linear(dim, m)
        self.fc2 = nn.Linear(m, m)
        
        self.fc3 = nn.Linear(m, m)
        self.fc4 = nn.Linear(m, m)

        
        self.outlayer = nn.Linear(m, 1)

    def forward(self, x):
        s = x@self.Ix
        y = self.fc1(x)
        y = F.relu(y**deg)
        y = self.fc2(y)
        y = F.relu(y**deg)
        y = y+s
        
        s=y
        y = self.fc3(y)
        y = F.relu(y**deg)
        y = self.fc4(y)
        y = F.relu(y**deg)
        y = y+s
        
        
        output = self.outlayer(y)
        return output
deg = 2

In [4]:
def sol_exact(x):
    x_tem = torch.exp(x)
    sol = torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return sol

In [5]:
def sol_exact_H1(x):
    x_tem = torch.exp(x)
    sol = torch.zeros(x.size()[0],dim_set+1).cuda()
    sol[:,0] = torch.sum(x_tem,dim=1)
    sol[:,1:dim_set+1] = torch.exp(x)
    return sol

In [6]:
def right(x):
    x_tem = torch.exp(x)
    f_tem = 2*torch.sum(x_tem,dim=1).reshape([x.size()[0],1])
    return f_tem 

In [7]:
dim_set = 8
Batch_size = 50000
def DGM():
    x = torch.rand(Batch_size,dim_set).cuda()
    x.requires_grad = True
    u = model(x)
    v= torch.ones(u.shape).cuda()
    du = torch.autograd.grad(u,x,grad_outputs=v,create_graph=True)[0]
    uxx = torch.zeros(Batch_size,dim_set).cuda()
    for i in range(dim_set):
        ux_tem = du[:,i].reshape([x.size()[0],1])
        uxx_tem = torch.autograd.grad(ux_tem,x,grad_outputs=v,create_graph=True)[0]
        uxx[:,i] = uxx_tem[:,i]
    loss1 = torch.sum((torch.sum(uxx,dim=1).reshape([x.size()[0],1]) + u - right(x))**2)/Batch_size
    bd_value= torch.zeros(100,1).cuda()
    v= torch.ones(bd_value.shape).cuda()
    for i in range(dim_set):
        notes1 = torch.rand(bd_value.shape[0],dim_set).cuda()
        notes2 = torch.rand(bd_value.shape[0],dim_set).cuda()
        notes1[:,i] = 0
        notes2[:,i] = 1
        notes1.requires_grad = True
        notes2.requires_grad = True
        v_bd1 = model(notes1)
        v_bd2 = model(notes2)
        Dbd1 = torch.autograd.grad(v_bd1,notes1,grad_outputs=v,create_graph=True)[0]
        Dbd2 = torch.autograd.grad(v_bd2,notes2,grad_outputs=v,create_graph=True)[0]
        bd_value = bd_value + (Dbd1[:,i]-1)**2+ (Dbd2[:,i]-math.exp(1))**2
    loss2 =   torch.sum(bd_value)/bd_value.shape[0]
    return loss1+loss2

In [8]:
model = ResNet(dim_set,20)

device=torch.device("cuda:0" )
model.to(device)

ResNet(
  (fc1): Linear(in_features=8, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=20, bias=True)
  (outlayer): Linear(in_features=20, out_features=1, bias=True)
)

In [9]:
notes = torch.rand(10000,dim_set).cuda()
exact = sol_exact_H1(notes)
u_L2 = sqrt(torch.sum((exact[:,0])**2)/10000)
Du_L2 = sqrt(torch.sum(exact[:,1:dim_set+1]**2)/10000)
def loss_error():
    notes = torch.rand(10000,dim_set).cuda()
    predict = torch.zeros(10000,2*dim_set+2).cuda()
    predict[:,0]= model(notes).reshape([1,notes.size()[0]])
    exact = sol_exact_H1(notes)
    value1 = sqrt(torch.sum((predict[:,0] - exact[:,0])**2)/10000)/u_L2
    return value1

In [10]:
traintime = 10000
error_save=np.zeros(traintime)
optimizer = optim.Adam([
                {'params': model.parameters()},
            ])
time_start=time.time()
for i in range(traintime):
    optimizer.zero_grad()
    losses1 = DGM()
    losses1.backward()
    optimizer.step()
    error1 = loss_error()
    error_save[i]=float(error1)
    #scheduler.step()
    if i%500==1:
        print("i= ",i)
        print("error =",error1)
        print("loss1 =",losses1.detach())
        np.save("error_save_DG_NBC_8D_v2.npy", error_save)
np.save("error_save_DG_NBC_8D_v2.npy", error_save)
time_end=time.time()
print('time cost',time_end-time_start,'s')

i=  1
error = 0.987975103286065
loss1 = tensor(7552.0846, device='cuda:0')
i=  501
error = 0.6051968809034348
loss1 = tensor(121.8562, device='cuda:0')
i=  1001
error = 0.30212224646269614
loss1 = tensor(27.0447, device='cuda:0')
i=  1501
error = 0.01498374892792749
loss1 = tensor(4.5711, device='cuda:0')
i=  2001
error = 0.010930091962291137
loss1 = tensor(6.3334, device='cuda:0')
i=  2501
error = 0.00892536374337051
loss1 = tensor(2.3112, device='cuda:0')
i=  3001
error = 0.009745563934135117
loss1 = tensor(1.9305, device='cuda:0')
i=  3501
error = 0.00783959020839369
loss1 = tensor(1.7300, device='cuda:0')
i=  4001
error = 0.0108202173999544
loss1 = tensor(5.4116, device='cuda:0')
i=  4501
error = 0.008375446230640841
loss1 = tensor(1.4728, device='cuda:0')
i=  5001
error = 0.008519064957657236
loss1 = tensor(1.4904, device='cuda:0')
i=  5501
error = 0.007486571579967056
loss1 = tensor(1.2257, device='cuda:0')
i=  6001
error = 0.008986883336954653
loss1 = tensor(1.5395, device='cuda

In [11]:
import matplotlib.pyplot as plt

In [12]:
index = np.arange(0,50000,10)
plt.plot(np.log10(error_save[0,index]))

IndexError: too many indices for array